## Data Cleaning with Python

by: Serhii Viskushenko

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# make plot show up without plt.show()
%matplotlib inline

# plot configurations
plt.rcParams["figure.figsize"] = (10, 8) 
plt.style.use('fivethirtyeight')

# date time library
import datetime as dt

ModuleNotFoundError: No module named 'seaborn'

## Helper functions

In [ ]:
def missing_cols(df):
    '''
    prints out columns with its amount of missing values with its %

    eg. column_A => 100 [10%]
    
    the above means column_A has 100 missing values, 10% of the entire column 
    '''

    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        # mean = sum / total
        pct = df[col].isnull().mean() * 100 
        if missing_vals != 0:
          print('{} => {} [{}%]'.format(col, df[col].isnull().sum(), round(pct, 2)))
        total += missing_vals
    
    if total == 0:
        print("no missing values left 🎉 ")

def get_memory(df):
  """ prints out size of data frame in MB """
  print(f"{round(df.memory_usage().sum() / 1000000, 2)} MB")

## Load data

In [ ]:
# data from github repo: https://github.com/dyvenia/data_cleaning
url = 'https://raw.githubusercontent.com/dyvenia/data_cleaning/main/dataset.csv'

df = pd.read_csv(url)

### data frame shape 


In [ ]:
df.shape # (no. of rows, no. of columns)

### data frame summary

In [ ]:
df.info()

### data frame data types

In [ ]:
df.dtypes

### First 5 rows of data

In [ ]:
df.head()

### First row of data

In [ ]:
df.head(n=1)

### Last five rows of data

In [ ]:
df.tail()

### Randomly sample rows

In [ ]:
df.sample(n = 5)

## Rename columns

In [ ]:
# show columns of dataframe
df.columns

In [ ]:
df.columns.to_list() # turn into list

### string methods

In [ ]:
# replace whitespace with underscore '_'
df.columns = df.columns.str.replace(' ', "_")

# lower case column names
df.columns = df.columns.str.lower() 

df.columns

### pandas rename

In [ ]:
# {'original_name' : 'new_name'}
new_names = {'name':'listing_name', 
             'latitude':'lat', 
             'longitude':'long'}

df.rename(columns = new_names).columns # only returning output

# without inplace=True, not updating our dataframe
df.columns

### inplace = True

- updates the data frame itself instead of returning an output

In [ ]:
df.rename(columns = new_names, inplace = True)
df.columns

A cool library for data cleaning is pyjanitor which provides easy implementations for many data cleaning tasks, check it out [here](https://pyjanitor.readthedocs.io/)

## Remove labels from values

- dollar sign -> Mixture of string and integer
- cause problems when filling in missing values or converting data types
- remove dollar sign and labels similar



In [ ]:
# grab the price column
df['price']

In [ ]:
df['price'] = df['price'].str.replace("$", "")
df['price'].head(3)

In [ ]:
# check price column data type
df['price'].dtype

## Converting data types

![dtypes](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F2000%2F1*wrXMq7iTWih7lsBBRQFxXg.png&f=1&nofb=1)

[Overview here](https://pbpython.com/pandas_dtypes.html)

### Converting to numeric

In [ ]:
df['price'] = pd.to_numeric(df['price'])
df['price'].dtype

In [ ]:
df.head(1)

In [ ]:
# view dtype of all columns
df.dtypes

### Convert to category

- `neighbourhood_group`
- `neighbourhood`
- `room_type`

Guidelines for conversion
- to reduce memory and increase performance with operations related to categorical data
- make sure data is clean before converting it

[More on this](https://pbpython.com/pandas_dtypes_cat.html)

In [ ]:
# check unique values in categorical column
df['neighbourhood_group'].unique()

In [ ]:
# show amount of categories in column
len(df['neighbourhood'].unique())

In [ ]:
df['room_type'].unique()

In [ ]:
get_memory(df)

In [ ]:
# for categorical data with small amounts of categories, converting them to 
# dtype category can save memory and make some operations more efficient 

df['room_type'] = df['room_type'].astype('category')

In [ ]:
df['room_type'].dtype

In [ ]:
df['room_type'].unique()

In [ ]:
get_memory(df)

### Convert to datetime 

In [ ]:
df['last_review'].head(3)

In [ ]:
df['last_review'] = pd.to_datetime(df['last_review'], format = "%Y-%m-%d")
df['last_review'].dtype

Refer to https://strftime.org/ for a list of date formats

In [ ]:
df.dtypes

In [ ]:
df['last_review'].head(3)

### Changing numeric types

ex: changing int types (int8 | int16 | int32 | int64)

- the numbers stands for bit
- int8 can store integers from -128 to 127.
- int16 can store integers from -32768 to 32767.
- int64 can store integers from -9223372036854775808 to 9223372036854775807.

In [ ]:
# get maximum from column
df['minimum_nights'].max()

In [ ]:
# get minimum from column
df['minimum_nights'].min()

In [ ]:
df['minimum_nights'].dtype

In [ ]:
get_memory(df)

In [ ]:
df['minimum_nights'] = df['minimum_nights'].astype('int16')
df['minimum_nights'].dtype

In [ ]:
get_memory(df)

## Missing data

### Checking which columns have missing data

In [ ]:
df.isnull().head()

### summing the missing values

In [ ]:
# True = 1, False = 0
df.isnull().sum()

### percentage of missing data

In [ ]:
missing_cols(df)

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cmap='viridis', cbar=False);

### How to deal with missing data?

Dealing with missing data is not simple task, you have need to consider why the data is missing in the first place, and domain knowledge to know what to impute. 

There also isn't a specific threshold for what percentage of missing data is accepted, it depends on the data.

If you mess it up, you will introduce bias to your data.  


#### Techniques 
1. Drop feature 
1. Drop the rows
1. Impute missing values (manually or automatically)

More information
- Read this [article](https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4) to go deeper into this topic

- A paper on "The prevention and handling of the missing data" ([Link](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3668100/))

In [ ]:
# copying original data set is good practice when you're experimenting with things
# so you don't have to run all the cells before when you mess up something

df_ori = df.copy()

#### Dropping columns with missing values

- usually worst strategy unless it has a lot of missing data (over 80 or 90%), or feature is not useful

In [ ]:
# Create list of columns you want to drop
colsToDrop = ['id','host_name','host_id']

# axis = 1 / 'columns' stands for column
# axis = 0 / 'index' stands for row
df.drop(colsToDrop, axis = 'columns', inplace = True)

In [ ]:
missing_cols(df)

In [ ]:
print(df.shape)

#### Remove rows with missing values
- Losing even more information (from other columns), so not the best method

In [ ]:
print(df.shape) 
print(df.dropna().shape)

#### Impute missing values


- A constant value related to the data, such as 0 for `number_of_reviews`, or "None" for `listing_name`.
- The value before or after the data point (backward fill, forward fill)
- Summary statistics such as mean, median or mode value for the column.
- A value estimated by algorithms or ML models like KNN.

> More detailed imputation techniques in this [article](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779)  

##### Imputing manually with pandas fillna

In [ ]:
df['reviews_per_month'].head(3)

In [ ]:
df['listing_name'].head(3)

In [ ]:
# {"column_name" : "value_to_replace"}
missing_vals_replace = {'reviews_per_month': 0,
                        'listing_name':'None'}

df.fillna(value = missing_vals_replace).head(3)['reviews_per_month']

In [ ]:
# mean of reviews_per_month
reviews_mean = df['reviews_per_month'].mean()
print(reviews_mean)

df.fillna(value = {'reviews_per_month' : reviews_mean}).head(3)['reviews_per_month']

In [ ]:
# median of reviews_per_month
reviews_median = df['reviews_per_month'].median()
print(reviews_median)

df.fillna(value = {'reviews_per_month' : reviews_median}).head(3)['reviews_per_month']

##### Imputing with bfill and ffill

bfill
- bfill stands for backwards fill
- means filling in missing value with value after it (fill it backwards)

ffill
- ffill stands for forward fill
- means filling in missing value with value before it (fill it forwards)


In [ ]:
df['reviews_per_month'].head()

In [ ]:
# imputing with bfill
df['reviews_per_month'].bfill().head()

In [ ]:
# imputing with ffill
df['reviews_per_month'].ffill().head()

In [ ]:
df['last_review'].head()

In [ ]:
df['last_review'].ffill().head()

In [ ]:
 # ffill entire data frame
 df.ffill().head(3)

In [ ]:
missing_vals_replace = {'reviews_per_month':0,
                        'listing_name':'None'}

df.fillna(missing_vals_replace, inplace = True)

missing_cols(df)

I leave date column empty because it makes more sense to leave it empty, and I don't want to introduce bias to it. this is a different case for time series data, as it's a must to impute missing values or analysis on the data won't work since it's cumulative

## Explode Date column

In [ ]:
df['last_review'].head()

In [ ]:
# Int64 are pandas data types that can handle missing values
df['year'] = df['last_review'].dt.year.astype('Int64')
df['month'] = df['last_review'].dt.month.astype('Int64')
df['day'] = df['last_review'].dt.day.astype('Int64')

In [ ]:
df.head(3)

## Data Inconsistencies

### out of range data

#### Are the values in the column `availability_365` within 365 ?

In [ ]:
# show statistics of a column
df['availability_365'].describe()

##### Assert keyword

In [ ]:
assert 1 + 1 == 2

In [ ]:
assert 1 + 1 == 3

In [ ]:
assert df['availability_365'].max() <= 365

In [ ]:
assert df['availability_365'].min() >= 0

#### Are lat and long coords valid coordinates?

- The latitude must be a number between -90 and 90 and the longitude between -180 and 180

Since you know the lat and long coords are for new york, you can ask the question whether the coords are within NYC itself

In [ ]:
df['lat'].describe()

In [ ]:
df['long'].describe()

The best way to make sure is to plot them out

Check out this [kaggle notebook](https://www.kaggle.com/dgomonov/data-exploration-on-nyc-airbnb/notebook) which does an analysis including plotting a new york city map containing these coordinates

### Categorical data inconsistency

- caused by human error (wrong spelling, different case categories)

In [ ]:
# show categories
df['neighbourhood_group'].unique()

In [ ]:
wrong_spelling = ['brookln', 'manhatan']
right_spelling = ['Brooklyn', 'Manhattan']

# replace the wrong spelling with right spelling
df['neighbourhood_group'].replace(to_replace = wrong_spelling, 
                                  value = right_spelling, 
                                  inplace = True)

# changing values removes the category date type
df['neighbourhood_group'].unique()

Usually it isn't so easy to deal with these value consistency since there could be hundreds of categories like the neighborhood column which has over 200 types, there are two ways to deal with this issue

1. Preprocess the text (lowercase, strip whitespace)
1. Use fuzzy matching to find similar words, and replace them. ([example](https://www.kaggle.com/rtatman/data-cleaning-challenge-inconsistent-data-entry/?scriptVersionId=3012975&cellId=14))

### Duplicate rows

In [ ]:
# are there any duplicate rows?
df.duplicated().any()

In [ ]:
# how many rows are duplicated?
df.duplicated().sum()

In [ ]:
# show me the duplicated row
df[df.duplicated()]

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.duplicated().any()

## Outliers

What?
- data point that is far from other observations in our data
- it arises from to erorrs in data collection or due to the influence of various factors on data
- when there are outliers which indicates erroneous or abnormal data then we can either remove them or correct them.

How to detect?
- with boxplots and histograms
- statistical methods like IQR, skewness, etc.

> To choose the best way to handle outliers one must have a good domain knowledge and information about where the data come from and what they mean ; it also depends on what analysis one is planning to perform.

More about data cleaning and outliers
- https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
- https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba 
- https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data
- https://aakada.wordpress.com/2019/02/24/how-to-deal-with-outliers-using-python-pandas/

### Continuous data

In [ ]:
df['price'].describe()

#### plotting a histogram

In [ ]:
df['price'].hist(bins=100);

#### Plotting a boxplot

![boxplot](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.simplypsychology.org%2Fboxplot-outliers.png&f=1&nofb=1)

In [ ]:
df.boxplot(column=['price']);

### Categorical data

In [ ]:
df['neighbourhood_group'].unique()

#### Plotting a bar plot



In [ ]:
# count of each category
df['neighbourhood_group'].value_counts()

In [ ]:
# plot a bar plot from value counts
df['neighbourhood_group'].value_counts().plot.bar();

In [ ]:
other_index = df['neighbourhood_group'].value_counts()[2:].index

# grab the indexes of the other categories, and tell pandas to change them 
# to 'Other'
df.loc[df['neighbourhood_group'].isin(other_index), 'neighbourhood_group'] = 'Other'

df['neighbourhood_group'].value_counts()

In [ ]:
df['neighbourhood_group'].value_counts().plot.bar();

In [ ]:
df['neighbourhood_group'] = df['neighbourhood_group'].astype('category')

## Save final cleaned dataset

In [ ]:
df.to_csv("clean_data")